In [3]:
import nibabel as nib
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.padding import PKCS7
import os

In [4]:
CT_SCANS = "../rawdata/ct_scans"
INFECTION_MASKS = "../rawdata/infection_mask"
LUNGANDINFECTION_MASKS = "../rawdata/lung_and_infection_mask"
LUNG_MASKS = "../rawdata/lung_mask"

In [5]:
ct_nii = nib.load(f"{CT_SCANS}/coronacases_org_001.nii")
image_data = ct_nii.get_fdata()

In [6]:
def encrypt_nifti(file_path, password, output_path):
    # Read the NIfTI image
    nifti_img = nib.load(file_path)
    data = nifti_img.get_fdata()
    
    # Serialize the data to bytes
    data_bytes = data.tobytes()

    # Derive a key from the password
    salt = os.urandom(16)  # Generate a random salt
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())  # Derive the key

    # Encrypt the data
    iv = os.urandom(16)  # Initialization vector
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    # Padding for block cipher
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(data_bytes) + padder.finalize()

    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted data and metadata
    with open(output_path, 'wb') as f:
        f.write(salt)  # Write salt for key derivation
        f.write(iv)    # Write IV for decryption
        f.write(encrypted_data)  # Write encrypted data

    print(f"Encrypted NIfTI image saved to {output_path}")

In [7]:
encrypt_nifti(f"{CT_SCANS}/coronacases_org_001.nii", "258456C@p", "encrypted.nii")

Encrypted NIfTI image saved to encrypted.nii


In [ ]:
def decrypt_nifti(encrypted_file_path, password, output_path):
    # Read the encrypted file
    with open(encrypted_file_path, 'rb') as f:
        salt = f.read(16)  # Read salt
        iv = f.read(16)    # Read IV
        encrypted_data = f.read()  # Read encrypted data

    # Derive the key from the password
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Decrypt the data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Remove padding
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    data_bytes = unpadder.update(padded_data) + unpadder.finalize()

    # Deserialize data
    nifti_img = nib.load(f"{CT_SCANS}/coronacases_org_001.nii")
    original_shape = nifti_img.shape
    data = np.frombuffer(data_bytes, dtype=np.float64).reshape(original_shape)

    # Save the decrypted NIfTI file
    decrypted_nifti = nib.Nifti1Image(data, nifti_img.affine, nifti_img.header)
    nib.save(decrypted_nifti, output_path)

    print(f"Decrypted NIfTI image saved to {output_path}")

In [10]:
decrypt_nifti("encrypted.nii", "258456C@p", "decrypted.nii")

FileNotFoundError: No such file or no access: '{CT_SCANS}/coronacases_org_001.nii'